# GSS Data Exploratory Analysis

## Objective
Explore the relationship between job satisfaction (SATJOB) and various psychological and work-related variables in the General Social Survey data.

### Variables of Interest:
- **Dependent Variable:** SATJOB (Job satisfaction)
- **Independent Variables:** 
  - HLTHDEP (Health depression)
  - FEELDOWN (Feeling down/depressed)
  - NOINTEREST (No interest in activities)
  - FEELNERV (Feeling nervous)
  - WORRY (Worry levels)
  - WRKMEANGFL (Work meaningfulness)
  - RICHWORK (Rich work experiences)
  - SATFIN (Financial satisfaction)
  - DISCAFFWNV (Discrimination affected workplace environment)

In [ ]:
# Import required libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np
import subprocess
import os
from pathlib import Path

# Configure plotly to render inline in the notebook
pio.renderers.default = "notebook_connected"

print("Libraries imported successfully")

## Data Loading

The GSS data has already been extracted from R format to CSV.

In [ ]:
# Load the GSS data directly from CSV
print("Loading GSS data...")

# Load with pandas and appropriate NA handling
gss_data = pd.read_csv(
    'data/gss_subset.csv',
    na_values=["NA", ""],
    on_bad_lines='skip',
    low_memory=False
)
print("Loaded with pandas")

print(f"Data loaded: {gss_data.shape[0]:,} rows, {gss_data.shape[1]} columns")
print(f"\nColumns: {list(gss_data.columns)}")
print(f"\nFirst few rows:")
print(gss_data.head())

## Data Exploration and Cleaning

In [ ]:
# Basic data info
print("Dataset Overview:")
print(f"Shape: {gss_data.shape}")
print(f"\nData types:")
print(gss_data.dtypes)

# Check for missing values
print("\nMissing values per column:")
missing_counts = gss_data.isna().sum()
print(missing_counts.to_frame().T)

# Calculate missing percentages
print("\nMissing Data Analysis:")
print("="*50)
total_rows = gss_data.shape[0]

# Missing data analysis with pandas
missing_data = (
    pd.DataFrame({
        'variable': gss_data.columns,
        'missing_count': missing_counts.values
    })
)
missing_data['total_rows'] = total_rows
missing_data['available_count'] = missing_data['total_rows'] - missing_data['missing_count']
missing_data['missing_pct'] = (missing_data['missing_count'] / missing_data['total_rows'] * 100).round(1)
missing_data['availability_pct'] = (missing_data['available_count'] / missing_data['total_rows'] * 100).round(1)
missing_data = missing_data.sort_values('missing_pct', ascending=False).reset_index(drop=True)

print(f"{'Variable':<15} {'Missing':<8} {'Available':<10} {'Missing %':<10} {'Available %':<12}")
print("-" * 65)
for _, row in missing_data.iterrows():
    print(f"{row['variable']:<15} {int(row['missing_count']):<8} {int(row['available_count']):<10} {row['missing_pct']:<10} {row['availability_pct']:<12}")

# Summary statistics
high_missing = (missing_data['missing_pct'] > 90).sum()
med_missing = (missing_data['missing_pct'] > 50).sum()
low_missing = (missing_data['missing_pct'] < 10).sum()
complete_data = (missing_data['missing_pct'] == 0).sum()

print(f"\nMissingness Summary:")
print(f"- Variables with >90% missing: {high_missing}")
print(f"- Variables with >50% missing: {med_missing}")
print(f"- Variables with <10% missing: {low_missing}")
print(f"- Variables with complete data: {complete_data}")

# Identify data quality tiers
high_quality = missing_data.loc[missing_data['missing_pct'] < 10, 'variable'].tolist()
medium_quality = missing_data.loc[(missing_data['missing_pct'] >= 10) & (missing_data['missing_pct'] < 50), 'variable'].tolist()
low_quality = missing_data.loc[missing_data['missing_pct'] >= 50, 'variable'].tolist()

print(f"\nData Quality Tiers:")
print(f"High quality (<10% missing): {high_quality}")
print(f"Medium quality (10-50% missing): {medium_quality}")
print(f"Low quality (≥50% missing): {low_quality}")

# Create visualization of missing data patterns
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=missing_data['variable'],
    y=missing_data['missing_pct'],
    name='Missing %',
    marker_color='red'
))

fig.update_layout(
    title="Missing Data Percentage by Variable",
    xaxis_title="Variables",
    yaxis_title="Missing Percentage (%)",
    xaxis_tickangle=-45,
    height=500
)
fig.show()

In [ ]:
# Focus on our variables of interest
target_vars = ['satjob', 'hlthdep', 'feeldown', 'nointerest', 'feelnerv', 'worry',
               'wrkmeangfl', 'richwork', 'satfin', 'discaffwnv']

existing_vars = [var for var in target_vars if var in gss_data.columns]
print(f"Variables available for analysis: {existing_vars}")

# Create subset with our variables + some demographics
analysis_vars = ['year', 'age', 'sex', 'race', 'degree'] + existing_vars
available_analysis_vars = [var for var in analysis_vars if var in gss_data.columns]

analysis_data = gss_data[available_analysis_vars]
print(f"\nAnalysis dataset shape: {analysis_data.shape}")
print(f"Variables: {list(analysis_data.columns)}")

# Analyze missingness patterns for our target variables
print(f"\nMissingness Analysis for Target Variables:")
print("="*55)

target_missing = {}
for var in existing_vars:
    missing_count = gss_data[var].isna().sum()
    total_count = gss_data.shape[0]
    missing_pct = (missing_count / total_count * 100)
    available_count = total_count - missing_count
    
    target_missing[var] = {
        'missing_count': int(missing_count),
        'available_count': int(available_count),
        'missing_pct': float(missing_pct)
    }
    
    print(f"{var:<12}: {missing_count:>6} missing ({missing_pct:>5.1f}%), {available_count:>6} available")

# Check when these variables were measured (by year)
print(f"\nVariable Availability by Year:")
print("="*35)

for var in existing_vars:
    years_with_data = gss_data.loc[gss_data[var].notna(), 'year'].dropna().unique()
    years_with_data = np.sort(years_with_data)
    
    if len(years_with_data) > 0:
        year_range = f"{years_with_data.min()}-{years_with_data.max()}"
        n_years = len(years_with_data)
        print(f"{var:<12}: {n_years:>2} years ({year_range})")
    else:
        print(f"{var:<12}: No data available")

# Identify which variables can be analyzed together
print(f"\nVariable Overlap Analysis:")
print("="*30)

satjob_overlaps = {}
for var in existing_vars:
    if var != 'satjob':
        overlap_count = (gss_data['satjob'].notna() & gss_data[var].notna()).sum()
        satjob_overlaps[var] = int(overlap_count)
        print(f"SATJOB + {var:<12}: {overlap_count:>5} complete pairs")

# Find the best candidates for analysis
viable_vars = [var for var, count in satjob_overlaps.items() if count > 1000]
print(f"\nViable variables for correlation analysis (>1000 pairs): {viable_vars}")

## Dependent Variable Analysis: Job Satisfaction (SATJOB)

In [ ]:
# Analyze SATJOB
print("Job Satisfaction (SATJOB) Analysis:")

# Use pandas for analysis
sat = gss_data['satjob'].dropna()

# Basic statistics using pandas
stats = {
    'count': float(sat.shape[0]),
    'mean': sat.mean(),
    'std': sat.std(),
    'min': sat.min(),
    '25%': sat.quantile(0.25),
    'median': sat.median(),
    '75%': sat.quantile(0.75),
    'max': sat.max()
}

print(f"\nBasic Statistics:")
for stat, value in stats.items():
    print(f"{stat}: {value:.6f}")

# Value counts using pandas
print(f"\nValue Counts:")
vc = gss_data['satjob'].dropna().value_counts().sort_index()
for level, count in vc.items():
    print(f"{level}: {count}")

# Visualization
if not vc.empty:
    fig = px.bar(x=vc.index.tolist(), 
                 y=vc.values.tolist(),
                 title="Distribution of Job Satisfaction (SATJOB)",
                 labels={'x': 'Job Satisfaction Level', 'y': 'Count'})
    fig.show()

## Independent Variables Analysis

In [ ]:
# Analyze each independent variable
independent_vars = ['hlthdep', 'feeldown', 'nointerest', 'feelnerv', 'worry',
                   'wrkmeangfl', 'richwork', 'satfin', 'discaffwnv']

available_indep_vars = [var for var in independent_vars if var in gss_data.columns]

print(f"Analyzing {len(available_indep_vars)} independent variables...")

print("\nBasic Statistics for Independent Variables:")
for var in available_indep_vars:
    print(f"\n{var.upper()}:")
    
    # Series with non-null values
    var_series = gss_data[var]
    var_data = var_series.dropna()
    
    # Check if numeric or categorical
    is_numeric = pd.api.types.is_numeric_dtype(var_series)
    
    if is_numeric and var_data.shape[0] > 0:
        # Numeric statistics
        print(f"count: {int(var_data.shape[0])}")
        print(f"mean: {var_data.mean()}")
        print(f"std: {var_data.std()}")
        print(f"min: {var_data.min()}")
        print(f"max: {var_data.max()}")
    else:
        # Categorical statistics
        print(f"count: {int(var_data.shape[0])}")
    
    # Missing values
    missing_count = gss_data[var].isna().sum()
    print(f"Missing values: {int(missing_count)}")
    
    # Show unique values and counts
    if var_data.shape[0] > 0:
        unique_vals = np.sort(var_data.unique())
        if len(unique_vals) <= 10:
            print(f"Unique values: {unique_vals.tolist()}")
            
            # Value counts
            value_counts = var_data.value_counts().sort_index()
            print(f"Value counts:")
            for value, count in value_counts.items():
                print(f"{value}: {count}")
    else:
        print("No data available for analysis")

## Correlation Analysis

In [ ]:
# Correlation analysis using pandas
print("Computing pairwise correlations with SATJOB...")
correlation_vars = ['satjob'] + available_indep_vars
print(f"Total observations: {gss_data.shape[0]}")

# Ensure SATJOB is numeric
satjob_numeric = pd.to_numeric(gss_data['satjob'], errors='coerce')

satjob_correlations = {}
for var in available_indep_vars:
    var_numeric = pd.to_numeric(gss_data[var], errors='coerce')
    pair = pd.DataFrame({'satjob': satjob_numeric, var: var_numeric}).dropna(subset=['satjob', var])
    n_obs = pair.shape[0]
    if n_obs > 10:
        corr_result = pair['satjob'].corr(pair[var])
        satjob_correlations[var] = {
            'correlation': float(corr_result),
            'n_obs': int(n_obs)
        }
        print(f"{var}: r = {corr_result:.3f} (n = {n_obs})")
    else:
        print(f"{var}: insufficient data (n = {n_obs})")

# Create visualization of correlations
if satjob_correlations:
    vars_list = list(satjob_correlations.keys())
    corr_values = [satjob_correlations[var]['correlation'] for var in vars_list]
    n_obs = [satjob_correlations[var]['n_obs'] for var in vars_list]

    fig = px.bar(x=vars_list, y=corr_values,
                 title="Correlations with Job Satisfaction (SATJOB)",
                 labels={'x': 'Variables', 'y': 'Correlation with SATJOB'},
                 hover_data={'n_obs': n_obs})
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

    print("\nAttempting correlation matrix for variables with sufficient data...")
    vars_with_data = ['satjob'] + [var for var in available_indep_vars 
                                   if var in satjob_correlations and 
                                   satjob_correlations[var]['n_obs'] > 1000]
    if len(vars_with_data) > 2:
        df_corr = gss_data[vars_with_data].apply(pd.to_numeric, errors='coerce')
        corr_matrix = df_corr.corr()
        fig = px.imshow(corr_matrix.values,
                        x=vars_with_data,
                        y=vars_with_data,
                        title="Correlation Matrix (Pairwise Complete Observations)",
                        color_continuous_scale='RdBu',
                        aspect='auto',
                        zmin=-1, zmax=1)
        fig.show()
        print(f"\nCorrelation matrix computed for {len(vars_with_data)} variables")
        print("Variables included:", vars_with_data)
    else:
        print("Not enough variables with sufficient data for correlation matrix")
else:
    print("No valid correlations could be computed")

## Relationship Visualizations

In [ ]:
# Create scatter plots for key relationships using pandas
print("Creating relationship visualizations...")

# Create subplots for each independent variable vs SATJOB
n_vars = len(available_indep_vars)
n_cols = min(3, n_vars)
n_rows = (n_vars + n_cols - 1) // n_cols

fig = make_subplots(rows=n_rows, cols=n_cols,
                    subplot_titles=[var.upper() for var in available_indep_vars])

sat_series = pd.to_numeric(gss_data['satjob'], errors='coerce')

for i, var in enumerate(available_indep_vars):
    row = i // n_cols + 1
    col = i % n_cols + 1
    
    # Clean data for this pair using pandas
    var_series = pd.to_numeric(gss_data[var], errors='coerce')
    mask = sat_series.notna() & var_series.notna()
    if mask.any():
        fig.add_trace(
            go.Scatter(x=var_series[mask], y=sat_series[mask],
                      mode='markers', name=var,
                      showlegend=False, opacity=0.6),
            row=row, col=col
        )

fig.update_layout(height=300*n_rows, 
                  title_text="Relationships between Independent Variables and Job Satisfaction")
fig.show()

## Summary Statistics by Groups

In [ ]:
# Group analysis by demographic variables using pandas
demographic_vars = ['sex', 'race', 'degree']
available_demo_vars = [var for var in demographic_vars if var in gss_data.columns]

if len(available_demo_vars) > 0:
    print("Job Satisfaction by Demographic Groups:")
    
    for var in available_demo_vars:
        print(f"\n{var.upper()}:")
        
        # Calculate group statistics using pandas
        df = gss_data[[var, 'satjob']].copy()
        df[var] = df[var].astype('category')
        df['satjob'] = pd.to_numeric(df['satjob'], errors='coerce')
        df = df.dropna(subset=[var, 'satjob'])
        
        if df.empty:
            print("No data available")
            continue
        
        group_stats = df.groupby(var)['satjob'].agg(['count', 'mean', 'std']).reset_index()
        group_stats['mean'] = group_stats['mean'].round(3)
        group_stats['std'] = group_stats['std'].round(3)
        
        # Display results
        print(f"{'Group':<8} {'Count':<8} {'Mean':<8} {'Std':<8}")
        print("-" * 32)
        for _, row in group_stats.iterrows():
            print(f"{row[var]:<8} {int(row['count']):<8} {row['mean']:<8} {row['std']:<8}")
        
        # Create box plot
        fig = px.box(df, x=var, y='satjob', title=f"Job Satisfaction by {var.upper()}")
        fig.update_xaxes(title=var.upper())
        fig.update_yaxes(title="Job Satisfaction")
        fig.show()
else:
    print("No demographic variables available for group analysis")

## Time Trends Analysis

In [ ]:
# Analyze trends over time using pandas
if 'year' in gss_data.columns:
    print("Time Trends Analysis:")
    
    # Prepare data
    df = gss_data[['year', 'satjob']].copy()
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['satjob'] = pd.to_numeric(df['satjob'], errors='coerce')
    df = df.dropna(subset=['year', 'satjob'])
    
    # Calculate mean satisfaction by year
    yearly_means = df.groupby('year')['satjob'].agg(['count', 'mean', 'std']).reset_index()
    yearly_means['mean'] = yearly_means['mean'].round(3)
    yearly_means['std'] = yearly_means['std'].round(3)
    yearly_means = yearly_means.sort_values('year')
    
    print("\nJob Satisfaction by Year (first 10 years):")
    print(f"{'Year':<8} {'Count':<8} {'Mean':<8} {'Std':<8}")
    print("-" * 32)
    for _, row in yearly_means.head(10).iterrows():
        print(f"{int(row['year']):<8} {int(row['count']):<8} {row['mean']:<8} {row['std']:<8}")
    
    # Plot time trend
    fig = px.line(yearly_means, x='year', y='mean',
                  title="Average Job Satisfaction Over Time",
                  labels={'year': 'Year', 'mean': 'Average Job Satisfaction'})
    fig.show()
    
    # Add trend analysis for key independent variables if available
    if len(available_indep_vars) > 0:
        print("\nCorrelations with Year (time trends):")
        for var in available_indep_vars:
            v = pd.to_numeric(gss_data[var], errors='coerce')
            y = pd.to_numeric(gss_data['year'], errors='coerce')
            mask = v.notna() & y.notna()
            if mask.sum() > 10:
                print(f"{var}: {y[mask].corr(v[mask]):.3f}")
            else:
                print(f"{var}: insufficient data")
else:
    print("Cannot perform time trends analysis - missing year variable")

## Summary and Conclusions

In [ ]:
# Summary of findings using pandas
print("EXPLORATORY DATA ANALYSIS SUMMARY")
print("="*50)

print(f"\n1. Dataset Overview:")
print(f"   - Total observations: {gss_data.shape[0]:,}")
print(f"   - Total variables: {gss_data.shape[1]}")
print(f"   - Variables of interest found: {len(existing_vars)}")

# Calculate SATJOB statistics using pandas
sat_series = pd.to_numeric(gss_data['satjob'], errors='coerce')
valid_count = sat_series.notna().sum()
missing_count = sat_series.isna().sum()
missing_pct = (missing_count / gss_data.shape[0]) * 100
mean_val = sat_series.mean()
std_val = sat_series.std()

print(f"\n2. Dependent Variable (SATJOB):")
print(f"   - Valid responses: {valid_count:,}")
print(f"   - Missing values: {missing_count:,}")
print(f"   - Missing percentage: {missing_pct:.1f}%")
print(f"   - Mean satisfaction: {mean_val:.3f}")
print(f"   - Standard deviation: {std_val:.3f}")

print(f"\n3. Missing Data Patterns:")
print(f"   - Most psychological variables (HLTHDEP, FEELDOWN, etc.) have >95% missing data")
print(f"   - These variables were likely asked only in specific survey years")
print(f"   - SATFIN and RICHWORK have better coverage (~6% and ~61% missing respectively)")
print(f"   - Complete case analysis across all variables is impossible (0 complete cases)")

print(f"\n4. Independent Variables:")
print(f"   - Available variables: {', '.join(available_indep_vars)}")
if 'viable_vars' in locals():
    print(f"   - Variables with sufficient data for correlation: {viable_vars}")

# Check if we have correlation results
if 'satjob_correlations' in locals() and satjob_correlations:
    print(f"\n5. Key Correlations with SATJOB:")
    sorted_corrs = sorted(satjob_correlations.items(), 
                         key=lambda x: abs(x[1]['correlation']), 
                         reverse=True)
    for var, corr_info in sorted_corrs[:5]:
        print(f"   - {var}: r = {corr_info['correlation']:.3f} (n = {corr_info['n_obs']})")

print(f"\n6. Data Quality Assessment:")
print(f"   - Survey design: Different modules asked in different years")
print(f"   - Temporal coverage: Data spans 1972-2024, but not all variables in all years")
print(f"   - Analysis limitations: Pairwise correlations only, no multivariate analysis possible")
print(f"   - Missing data mechanism: Likely Missing Completely At Random (MCAR) due to survey design")

print(f"\n7. Recommendations for Further Analysis:")
print(f"   - Focus on variables with sufficient data (SATFIN, RICHWORK)")
print(f"   - Analyze subsets of data by survey year or wave")
print(f"   - Consider multiple imputation for key psychological variables")
print(f"   - Investigate which years psychological measures were included")
print(f"   - Use GSS panel data for longitudinal analysis of available variables")
print(f"   - Consider external validation with other datasets")

print(f"\n8. Alternative Analysis Strategies:")
print(f"   - Year-specific analysis for psychological variables")
print(f"   - Propensity score matching to handle missing data")
print(f"   - Sensitivity analysis with different missing data assumptions")
print(f"   - Focus on demographic predictors which have better coverage")

print("\nAnalysis complete!")